In [5]:
!pip install -U git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /private/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/pip-req-build-hd3xs9jj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/pip-req-build-hd3xs9jj
  Resolved https://github.com/huggingface/transformers to commit d5d007a1a0f0c11a726a54c8f00bd71825f84d02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11430792 sha256=49d9121a8ce51eef50627b6d0f6e5ae92b790dbd63c300c344534285d9cfc208
  Stored in directory: /private/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/pip-ephem-wheel-cache-obbxp31f/wheels/49/a7/50/c9fdabbf10e51bb1256adb0c1a587fedd7184f5bad28d47fe3
Successfully built transformers
  Attempting uninstall: transformers
    Found existing instal

In [3]:
!pip install --upgrade transformers

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForVideoClassification

tokenizer = AutoTokenizer.from_pretrained("facebook/vjepa2-vitl-fpc32-256-diving48")
model = AutoModelForVideoClassification.from_pretrained(
    "facebook/vjepa2-vitl-fpc32-256-diving48"
)

ValueError: The checkpoint you are trying to load has model type `vjepa2` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [7]:
import torch
import numpy as np

from torchcodec.decoders import VideoDecoder
from transformers import AutoVideoProcessor, AutoModelForVideoClassification

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load model and video preprocessor
hf_repo = "facebook/vjepa2-vitl-fpc32-256-diving48"

model = AutoModelForVideoClassification.from_pretrained(hf_repo).to(device)
processor = AutoVideoProcessor.from_pretrained(hf_repo)

# To load a video, sample the number of frames according to the model.
# For this model, we use 64.
video_url = "https://huggingface.co/datasets/nateraw/kinetics-mini/resolve/main/val/bowling/-WH-lxmGJVY_000005_000015.mp4"
vr = VideoDecoder(video_url)
frame_idx = np.arange(
    0, model.config.frames_per_clip, 2
)  # you can define more complex sampling strategy
video = vr.get_frames_at(indices=frame_idx).data  # frames x channels x height x width

# Preprocess and run inference
inputs = processor(video, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits

print("Top 5 predicted class names:")
top5_indices = logits.topk(5).indices[0]
top5_probs = torch.softmax(logits, dim=-1).topk(5).values[0]
for idx, prob in zip(top5_indices, top5_probs):
    text_label = model.config.id2label[idx.item()]
    print(f" - {text_label}: {prob:.2f}")

ValueError: The checkpoint you are trying to load has model type `vjepa2` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [9]:
import torch
from torchcodec.decoders import VideoDecoder
import numpy as np

processor = AutoVideoProcessor.from_pretrained("facebook/vjepa2-vitl-fpc64-256")
model = AutoModel.from_pretrained(
    "facebook/vjepa2-vitl-fpc64-256",
    torch_dtype=torch.float16,
    device_map="auto",
    attn_implementation="sdpa",
)

video_url = "https://huggingface.co/datasets/nateraw/kinetics-mini/resolve/main/val/archery/-Qz25rXdMjE_000014_000024.mp4"

vr = VideoDecoder(video_url)
frame_idx = np.arange(
    0, 64
)  # choosing some frames. here, you can define more complex sampling strategy
video = vr.get_frames_at(indices=frame_idx).data  # T x C x H x W
video = processor(video, return_tensors="pt").to(model.device)
outputs = model(**video)

# V-JEPA 2 encoder outputs, same as calling `model.get_vision_features()`
encoder_outputs = outputs.last_hidden_state

# V-JEPA 2 predictor outputs
predictor_outputs = outputs.predictor_output.last_hidden_state

ValueError: Unrecognized video processor in facebook/vjepa2-vitl-fpc64-256. Should have a `video_processor_type` key in its video_preprocessor_config.json of config.json, or one of the following `model_type` keys in its config.json: instructblip, instructblipvideo, internvl, llava_next_video, llava_onevision, qwen2_5_omni, qwen2_5_vl, qwen2_vl, smolvlm, video_llava

In [13]:
!pip install -q git+https://github.com/huggingface/transformers@v4.52.4-VJEPA-2-preview

In [1]:
import torch
from torchcodec.decoders import VideoDecoder
from transformers import AutoVideoProcessor, AutoModelForVideoClassification
from transformers import AutoVideoProcessor, AutoModel
import numpy as np

processor = AutoVideoProcessor.from_pretrained("facebook/vjepa2-vitl-fpc64-256")
model = AutoModel.from_pretrained(
    "facebook/vjepa2-vitl-fpc64-256",
    torch_dtype=torch.float16,
    device_map="mps",
    attn_implementation="sdpa",
)

video_url = "https://huggingface.co/datasets/nateraw/kinetics-mini/resolve/main/val/archery/-Qz25rXdMjE_000014_000024.mp4"

vr = VideoDecoder(video_url)
frame_idx = np.arange(
    0, 64
)  # choosing some frames. here, you can define more complex sampling strategy
video = vr.get_frames_at(indices=frame_idx).data  # T x C x H x W
video = processor(video, return_tensors="pt").to(model.device)
outputs = model(**video)

# V-JEPA 2 encoder outputs, same as calling `model.get_vision_features()`
encoder_outputs = outputs.last_hidden_state

# V-JEPA 2 predictor outputs
predictor_outputs = outputs.predictor_output.last_hidden_state

/Users/manojkl/miniconda3/envs/llmvision/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: MPS backend out of memory (MPS allocated: 13.64 GB, other allocations: 4.02 GB, max allowed: 18.13 GB). Tried to allocate 1024.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [2]:
import torchcodec

print(dir(torchcodec))

['AudioSamples', 'Frame', 'FrameBatch', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_core', '_frame', '_internally_replaced_utils', 'decoders', 'samplers', 'version']


In [3]:
import torchcodec

print(torchcodec.__file__)

/Users/manojkl/miniconda3/envs/llmvision/lib/python3.12/site-packages/torchcodec/__init__.py


In [7]:
!cp -r /Users/manojkl/miniconda3/envs/llmvision/lib/python3.12/site-packages/torchcodec /Users/manojkl/Documents/new